## Об этом Notebook
В этом Notebook вы познакомитесь:
- С функциями агрегации в Pandas для Series и для Dataframe
- Как можно объединять несколько датафреймов в один
- Посмотрим как работает функция Pivot в Pandas

***

In [ ]:
# Выполни прежде чем проходить Notebook
from google.colab import drive
import os
drive.mount ('/content/gdrive', force_remount=True)

%run /content/gdrive/MyDrive/02_NumPy_and_Pandas/0_package_installation.ipynb
# Перезагрузка ядра
os.kill(os.getpid(), 9)

Mounted at /content/gdrive
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 24.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 21.7 MB/s eta 0:00:00
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.3/98.3 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: matplotlib
    Found existing installation:

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 35.1 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.23.5
    Uninstalling numpy-1.23.5:
      Successfully uninstalled numpy-1.23.5
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
numba 0.56.4 requires numpy<1.24,>=1.18, but you have numpy 1.25.2 which is incompatible.
tensorflow 2.12.0 requires numpy<1.24,>=1.22, but you have numpy 1.25.2 which is incompatible.


In [ ]:
import pandas as pd
import numpy as np
data = pd.read_csv('/content/gdrive/MyDrive/02_NumPy_and_Pandas/Data/avocado.csv')
data = data.loc[:, ['Date', 'AveragePrice', 'type', 'year', 'region', 'Total Volume', 'Large Hass Avocado', 'Small Bags', 'XLarge Bags']]

Описание данных стоимости и продаж авакадо на нескольких рынках в США.

- Date: дата наблюдения
- AveragePrice: средняя стоимость 1-ого авакадо
- type: тип авакадо
- year: год
- Region: город или регион наблюдения
- Total Volume: общее кол-во проданных авакадо
- Large Hass Avocado: вес Хасс авокадо в килограммах

## 1. Подсчет статистик у серий

Давайте посмотрим как посчитать агрегации  `sum()`, `mean()`, `median()`, `max()` и `min()` в серии Pandas.

![рисунок](https://drive.google.com/uc?id=1pUWzBwDWOtFusN--dwTPQV4JxHriJMPE)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

Прошу заметить, что NaN значения при подсчетах статистик **не участвуют**!

In [ ]:
our_series = pd.Series([8.45, 3.15, 1.25, 10.55, 2.40])

print("Суммируем все числа серии и получается: ",our_series.sum().round())
print("Считаем среднее у серии и получается: ",our_series.mean().round())
print("Считаем медиану у серии и получается: ",our_series.median().round())
print("Считаем максимум у серии и получается: ",our_series.max().round())
print("Считаем минимум у серии и получается: ",our_series.min().round())

Суммируем все числа серии и получается:  26.0
Считаем среднее у серии и получается:  5.0
Считаем медиану у серии и получается:  3.0
Считаем максимум у серии и получается:  11.0
Считаем минимум у серии и получается:  1.0


Агрегируем используя функцию `groupby()`

In [ ]:
s = pd.Series([1, 2, 10, 15, 19, 27], index=['girls', 'boys', 'boys', 'girls', 'boys', 'girls'])
print(s)
print("\n")
# Посчитаем в разрезе девочек и мальчиков разные агрегаты с помощью groupby
print("Среднее: \n",s.groupby(by=s.index).mean())
print("Максимум: \n",s.groupby(by=s.index).max())
print("Минимум: \n",s.groupby(by=s.index).min())

girls     1
boys      2
boys     10
girls    15
boys     19
girls    27
dtype: int64


Среднее: 
 boys     10.333333
girls    14.333333
dtype: float64
Максимум: 
 boys     19
girls    27
dtype: int64
Минимум: 
 boys     2
girls    1
dtype: int64


## 2. Подсчет статистик у одного измерения в Pandas DataFrame

Давайте посмотрим на пример, если мы хотим посчитать среднюю цену авакадо в зависимости от его типа. Сначала разделим наш dataframe на две части на organic и на conventional

In [ ]:
# Создаем булевый массив используя совпадения по названию типа
filter_o = data['type'] == 'organic'
filter_o

0         True
1         True
2         True
3        False
4         True
         ...  
18244    False
18245    False
18246     True
18247     True
18248    False
Name: type, Length: 18249, dtype: bool

In [ ]:
# Отбираем все записи и колонки по булевому массиву
data_organic = data.loc[filter_o]
data_organic

,Date,AveragePrice,type,year,region,Total Volume
0,2015-01-04,1.75,organic,2015,Southeast,27365.89
1,2015-01-04,1.49,organic,2015,Chicago,17723.17
2,2015-01-04,1.68,organic,2015,HarrisburgScranton,2896.72
4,2015-01-04,1.64,organic,2015,Boise,1505.12
7,2015-01-04,1.50,organic,2015,LasVegas,6329.83
...,...,...,...,...,...,...
18240,2018-03-25,1.75,organic,2018,Jacksonville,5518.73
18241,2018-03-25,1.42,organic,2018,RichmondNorfolk,17340.49
18243,2018-03-25,1.74,organic,2018,Boston,38441.23
18246,2018-03-25,1.42,organic,2018,SouthCentral,163496.70


### Задача 5.2.1:
1. Отфильтруйте датафрейм  data только по тем записям, где type == "conventional", используя булев массив

In [ ]:
# Напишите свой код ниже:


Теперь давайте посчитаем среднюю цену для обоих типов авакад, используя функцию `.mean()`

In [ ]:
avg_organic = data_organic['AveragePrice'].mean()
avg_organic

1.6539986846432093

### Задача 5.2.2:
1. Аналогично посчитайте среднюю цену для авакад с типом conventional.
2. Результат вычислений положите в переменную **avg_conventional**

In [ ]:
# Напишите свой код ниже:


In [ ]:
# Давайте посмотрим на полученные результаты
print(avg_organic, avg_conventional)
print('\n')
print(type(avg_organic), type(avg_conventional))

1.6539986846432093 1.1580396668858206


<class 'numpy.float64'> <class 'numpy.float64'>


Полученный результат мы можем объединить в единый DataFrame, используя следующий код:

In [ ]:
data_output = pd.DataFrame({'Type':['organic','conventional'],
                            'Average_price':[avg_organic, avg_conventional]})

In [ ]:
# Полученный датафрейм
print('\nResult dataframe :\n',data_output)


Result dataframe :
            Type  Average_price
0       organic       1.653999
1  conventional       1.158040


## 3. Подсчет статистик у двух измерений в Pandas DataFrame

Теперь посмотрим как `groupby()` работает с двумя и более столбцами у датафреймов

![рисунок](https://drive.google.com/uc?id=17YhfS3QD-UsEVDTiMRQ6yBCJzG-TiZ6x)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

Функция `groupby()` позволяет нам быстро разделить датафрейм на разные группы после чего можем посчитать любые агрегаты. В результате преобразований от `groupby()` на выходе мы получаем объект `DataFrameGroupBy`. Никаких вычислений не произойдет, если мы не укажем после `groupby()` функцию для агрегации, например, `mean()`, `sum()` и т.д.

In [ ]:
# Как видите на выходе мы не получаем расчеты
data.groupby('year')

Теперь давайте добавим функцию `mean()`, чтобы посчитать среднюю цену авакадо в зависимости от его типа

In [ ]:
by_type_total = data.groupby('type')['AveragePrice'].mean()
print(by_type_total)

type
conventional    1.158040
organic         1.653999
Name: AveragePrice, dtype: float64


In [ ]:
# Если мы хотим еще разбить по годам, то укажем еще и year в groupby
by_type_year = data.groupby(['type','year'])['AveragePrice'].mean()
print(by_type_year)

type          year
conventional  2015    1.077963
              2016    1.105595
              2017    1.294888
              2018    1.127886
organic       2015    1.673324
              2016    1.571684
              2017    1.735521
              2018    1.567176
Name: AveragePrice, dtype: float64


### Задача 5.3.1
1. Сколько килограммов Хасс авокадо было продано в разрезе типа, года и региона?
2. Результат положите в переменную by_year

In [ ]:
# Напишите свой код ниже:


# 4. Функцияя agg в Pandas Dataframe

Функция `agg()` - это сокращение от aggregate, данная функция используется для передачи функции или списка функций, которые должны быть применены к серии или даже к каждому элементу серии отдельно

Можно указывать в таком формате:
`our_dataset.agg({'First_column' : ['max', 'min'], 'Second_column' : ['mean', 'median']})`

In [ ]:
# Посчитаем максимум и минимум колонки Total Volume и среднее, минимум для колонки Small Bags, используя функцию .agg()
data.agg({'Total Volume' : ['max', 'min'], 'Small Bags' : ['min', 'mean']})

,Total Volume,Small Bags
max,62505646.52,NaN
min,84.56,0.000000
mean,NaN,182194.686696


Мы можем указывать `.agg()` также уже и для сгруппированных объектов, чтобы посчитать какие-нибудь статистики для выбранных колонок

![](https://drive.google.com/uc?id=174gO0DloNt3BoO4hBa82TPe5X0MdnlSa)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

In [ ]:
# Сгруппируем данные по колонкам region и type
# Посчитаем минимум, максимум, среднее по средней цене AveragePrice
grouped = data.groupby(['region', 'type']).agg({'AveragePrice': ['min', 'max', 'mean']})
grouped

AveragePrice                
                                          min   max      mean
region              type                                     
Albany              conventional         0.85  1.80  1.348757
                    organic              1.32  2.13  1.773314
Atlanta             conventional         0.68  1.64  1.068817
                    organic              0.62  2.75  1.607101
BaltimoreWashington conventional         0.95  1.86  1.344201
...                                       ...   ...       ...
TotalUS             organic              1.00  2.09  1.546036
West                conventional         0.66  1.62  0.985089
                    organic              0.90  2.52  1.559349
WestTexNewMexico    conventional         0.52  1.22  0.842130
                    organic              1.04  2.93  1.688855

[108 rows x 3 columns]

В функцию `agg()` можно передавить свои собственно написанные функции:

In [ ]:
# Напишем функцию, которая считает 95% перцентиль для конкретной колонки, используя встроенную функцию .quantile(0.95)
def percentile_95(column):
    return column.quantile(0.95)

### Задача 5.4.1

1. Посчитайте 95% перцентиль для колонок 'Small Bags','Large Bags','XLarge Bags', используя функцию `agg()`
2. Воспользуйтесь кастомной функцией percentile_95, что мы объявили ячейкой выше

In [ ]:
# Начни писать свой код ниже


### Задача 5.4.2:

`groupby()` - может быть полезен, если мы хотим посмотреть на пропорцию авакадо в зависимости от его типа. Например, 97%  conventional и 3% organic.
Чтобы добиться этого результата:
1. Нам нужно сгруппироваться по колонке type, посчитать кол-во по Total Volume и положить результат в переменную volume_by_type
2. Разделить volume_by_type на сумму всех авакадо и запомнить результат в переменную proportion
3. Распечатать результат proportion помноженный на 100, чтобы сконвертировать результат в проценты

In [ ]:
# Посчитайте кол-во Total Volume, предварительно сгруппировав данные по полю type
# Начни писать свой код ниже:


type
conventional    9126
organic         9123
Name: Total Volume, dtype: int64

In [ ]:
# Получи пропорцию в зависимости от типа авакадо
# Начни писать свой код ниже:


In [ ]:
# Распечатай результат домножив на 100


type
conventional    50.00822
organic         49.99178
Name: Total Volume, dtype: float64


## 5. Объединение датафреймов в Pandas


Если вы знакомы с SQL, то операторы INNER JOIN, LEFT/RIGH JOIN, OUTER JOIN должны быть вам знакомы. Чтобы освежить в памяти общую концепию посмотрите на картинку ниже:
![](https://drive.google.com/uc?id=1nFUB5pDvUx2-wmG5tum7NFOmM_m-tyvq)

**Источник:** https://www.dataquest.io/blog/sql-joins-interview-questions-and-answers/

Pandas имеет 4 способа объединять датафреймы между собой. Разберем только функцию `Merge`.
Со всеми остальными можно познакомиться в оф. документации [здесь](https://pandas.pydata.org/docs/user_guide/merging.html).

### Merge

![](https://drive.google.com/uc?id=1RlaQmwAK9Pv7PXUkOIHRIthPpUxTWpu7)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

In [ ]:
# Создадим первый датафрейм
data_1 = pd.DataFrame({'key':['A','B','C','B','E','F','A','H','A','J'],
                      'values_1': range(10)})
print(data_1)

  key  values_1
0   A         0
1   B         1
2   C         2
3   B         3
4   E         4
5   F         5
6   A         6
7   H         7
8   A         8
9   J         9


In [ ]:
# Создадим второй датафрейм
data_2 = pd.DataFrame({'key':['A','B','C'],
                       'values_2':range(3)})
print(data_2)

  key  values_2
0   A         0
1   B         1
2   C         2


Наш первый датафрейм содержит несколько значений с ключом **A** и **B**, когда во втором датафрейме у нас лишь по 1 значению на каждую пару ключ-значение:

In [ ]:
# Merge data_1 и data_2
pd.merge(data_1, data_2)

,key,values_1,values_2
0,A,0,0
1,A,6,0
2,A,8,0
3,B,1,1
4,B,3,1
5,C,2,2


Так как колонка key присутствует в обоих датафреймах, то нам не пришлось указывать по какому ключу необходимо соединяться, В функции **merge** есть параметр `on`, в котором можно указать по какому ключу необходимо заджоиниться.

In [ ]:
# Merge data_1 и data_2, указав ключ явно
pd.merge(data_1, data_2, on = 'key')

,key,values_1,values_2
0,A,0,0
1,A,6,0
2,A,8,0
3,B,1,1
4,B,3,1
5,C,2,2


Также у merge есть параметр `how`, в котормо указывается какой именно JOIN нужно применить:
- `how = 'left'` = LEFT JOIN
- `how = 'inner'` = INNER JOIN - по умолчанию
- `how = 'right'` = RIGHT JOIN
- `how = 'outer'` = OUTER JOIN


In [ ]:
# LEFT JOIN
pd.merge(data_1, data_2, on = 'key', how = 'left')

,key,values_1,values_2
0,A,0,0.0
1,B,1,1.0
2,C,2,2.0
3,B,3,1.0
4,E,4,NaN
5,F,5,NaN
6,A,6,0.0
7,H,7,NaN
8,A,8,0.0
9,J,9,NaN


Если в датафреймах названия колонок не совпадают, то для такой ситуации есть параметр `left_on` и `right_on`.
- `left_on` ключ для датафрейма слева
- `right_on` ключ для датафрейма справа

In [ ]:
data_3 = pd.DataFrame({'key_left': ['E','F','G','H','I','J'],
                       'values': range(6)})
print(data_3)

  key_left  values
0        E       0
1        F       1
2        G       2
3        H       3
4        I       4
5        J       5


In [ ]:
data_4 = pd.DataFrame({'key_right': ['D','E','F','G'],
                       'values_2': range(4)})
print(data_4)

  key_right  values_2
0         D         0
1         E         1
2         F         2
3         G         3


In [ ]:
pd.merge(data_3, data_4, left_on= 'key_left', right_on= 'key_right', how = 'inner')

,key_left,values,key_right,values_2
0,E,0,E,1
1,F,1,F,2
2,G,2,G,3


Если нужно заджоинится используя более одного ключа, то необходимо указать ключи соединения в формате питоновского списка:

In [ ]:
df_3 = pd.DataFrame({'key_1':['apple','banana','coconut','pineapple','strawberry'],
                     'key_2':['yes','maybe','maybe','yes','no'],
                     'values_1': range(5)})
print(df_3)

        key_1  key_2  values_1
0       apple    yes         0
1      banana  maybe         1
2     coconut  maybe         2
3   pineapple    yes         3
4  strawberry     no         4


In [ ]:
df_4 = pd.DataFrame({'key_1':['apple','banana','coconut','strawberry','strawberry'],
                     'key_2':['no','maybe','yes','no','no'],
                     'values_1': range(5)})
print(df_4)

        key_1  key_2  values_1
0       apple     no         0
1      banana  maybe         1
2     coconut    yes         2
3  strawberry     no         3
4  strawberry     no         4


In [ ]:
pd.merge(df_3, df_4, on = ['key_1', 'key_2'], how = 'inner')

,key_1,key_2,values_1_x,values_1_y
0,banana,maybe,1,1
1,strawberry,no,4,3
2,strawberry,no,4,4


### Задача 5.5.1
1. Соедините датафрейм data_1, data_2 по ключу, по right join.

In [ ]:
# Напишите свой код ниже:


### Задача 5.5.2
1. Соедините датафрейм data_1, data_2 по ключу, по outer join.

In [ ]:
# Напишите свой код ниже:


## 6. Сводные таблицы с Pivot
**PIVOT** - преобразует датафрейм в таблицу, где значения использованных колонок становятся новыми индексами и колонками.

![](https://drive.google.com/uc?id=10DYbaDinOXbKlywylWT6rUMkaG512QlG)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

### 6.1 Функция Pivot_table
Всё как и в предыдущей функции, только можно произвести агрегацию, получая одно значение из группы с одинаковыми значениями в новых индексах и колонках. По умолчанию берётся среднее от группы значений.

![](https://drive.google.com/uc?id=17hbUITSTEx9IFNe4puFk-s4_66Wy78Z9)

**Источник:** https://betterprogramming.pub/pandas-illustrated-the-definitive-visual-guide-to-pandas-c31fa921a43

In [ ]:
df = pd.DataFrame({'Student Names' : ['Jenny', 'Singh', 'Charles', 'Richard', 'Veena'],
                   'Category' : ['Online', 'Offline', 'Offline', 'Offline', 'Online'],
                   'Gender' : ['Female', 'Male', 'Male', 'Male', 'Female'],
                  'Courses': ['Java', 'Spark', 'PySpark','Hadoop','C'],
                   'Fee': [15000, 17000, 27000, 29000, 12000],
                   'Discount': [1100, 800, 1000, 1600, 600]})

In [ ]:
p_table = pd.pivot_table(df, index=['Gender'])
p_table

/var/folders/h2/rfmmc6tj25z0qcd2yx8m2wsh0000gn/T/ipykernel_84099/1079762339.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  p_table = pd.pivot_table(df, index=['Gender'])


,Discount,Fee
Gender,,
Female,850.000000,13500.000000
Male,1133.333333,24333.333333
